In [1]:
ALGORITHM = 'Linear Regression'
ALGORITHM_DETAIL = 'random search'
VERSION = '03'

RANDOM_STATE = 101
TRAINING_SIZE = 0.9

CROSS_VALIDATION_SCORING = 'r2'

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import Lasso, Ridge

pipe = Pipeline([
    #('mms', MinMaxScaler()),
    ('std_scaler', StandardScaler()),
    ('model', Ridge())
])

In [3]:
from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np
from pandas import DataFrame
import math
from termcolor import colored

confirm_colab = False

try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = confirm_colab

if not IN_COLAB:
    from functions_20221018 import set_csv_directory, get_combined_dataset, add_supplements, tidy_dataset, feature_engineer

set_csv_directory('final_split')

debug_mode = False

In [4]:
#cutdown_rows = 1000
cutdown_rows = 0

LABEL = 'Price'

booleans = []
floats = ['location.latitude', 'location.longitude', 'bedrooms', 'bathrooms', 'nearestStation', 'latitude_deviation',
          'latitude_deviation2', 'longitude_deviation', 'longitude_deviation2']
floats = ['bedrooms', 'bathrooms', 'nearestStation', 'latitude_deviation2', 'longitude_deviation2']
categories = ['tenure.tenureType']

columns = []
columns.extend(booleans)
columns.extend(floats)
columns.extend(categories)

print(colored(f"features", "blue"), "-> ", columns)
columns.insert(0, LABEL)
print(colored(f"label", "green", None, ['bold']), "-> ", LABEL)

features ->  ['bedrooms', 'bathrooms', 'nearestStation', 'latitude_deviation2', 'longitude_deviation2', 'tenure.tenureType']
label ->  Price


In [5]:
filename = f'df_listings_v{VERSION}.csv'
remote_pathname = f'https://raw.githubusercontent.com/jayportfolio/capstone_streamlit/main/data/final/{filename}'
df_pathname_raw = f'../../data/source/{filename}'
df_pathname_tidy = f'../../data/final/{filename}'


def get_source_dataframe(rows=cutdown_rows, folder_prefix='../'):
    retrieval_type = None

    if IN_COLAB:
        inDF = pd.read_csv(remote_pathname, on_bad_lines='error', index_col=0)
        retrieval_type = 'tidy'
        print('loaded data from', remote_pathname)
    else:
        try:
            inDF = pd.read_csv(df_pathname_tidy, on_bad_lines='error', index_col=0)
            retrieval_type = 'tidy'
            print('loaded data from', df_pathname_tidy)
        except:
            try:
                inDF = pd.read_csv(df_pathname_raw, on_bad_lines='error', index_col=0)
                retrieval_type = 'raw'
                print(f'starting to get {retrieval_type} data...')
                print('loaded data from', df_pathname_raw)
                print(f'finished getting {retrieval_type} data!')
            except:
                print(f"WARNING: Failed to retrieved stored data for version {VERSION}, creating new source data.")
                retrieval_type = 'scratch'
                print(f'starting to get {retrieval_type} data...')
                inDF = get_combined_dataset(HOW='inner', early_duplicates=True, folder_prefix=folder_prefix)
                print(f'finished getting {retrieval_type} data!')

                print(f'starting to save {retrieval_type} data...')
                inDF.to_csv(df_pathname_raw)
                print(f'finished saving {retrieval_type} data!')

    if rows and rows > 0:
        inDF = inDF[:rows]
    return inDF, retrieval_type


def create_train_test_data(df_orig, return_index=False, drop_nulls=True):
    df = df_orig.copy()

    if drop_nulls:
        df.dropna(inplace=True)

    if return_index:
        df.reset_index(inplace=True)

    for column in categories:
        df = pd.concat([df, pd.get_dummies(df[column], prefix=column)], axis=1)
        df.drop([column], axis=1, inplace=True)  # now drop the original column (you don't need it anymore),

    ins = df.pop('index')
    df.insert(1, 'index2', ins)
    df.insert(0, 'index', ins)

    #features = df[df.columns[1:]].values
    features = df[df.columns[2:]].values
    #labels = df[LABEL].values
    labels = df.iloc[:, 0:2].values

    if not return_index:
        return train_test_split(features, labels, train_size=0.9, random_state=RANDOM_STATE)
    else:
        X_train1, X_test1, y_train1, y_test1 = train_test_split(features, labels, train_size=0.9,
                                                                random_state=RANDOM_STATE)
        X_train_index = X_train1[:, 0].reshape(-1, 1)
        y_train_index = y_train1[:, 0].reshape(-1, 1)
        X_test_index = X_test1[:, 0].reshape(-1, 1)
        y_test_index = y_test1[:, 0].reshape(-1, 1)
        X_train1 = X_train1[:, 1:]
        y_train1 = y_train1[:, 1].reshape(-1, 1)
        X_test1 = X_test1[:, 1:]
        y_test1 = y_test1[:, 1].reshape(-1, 1)

        return X_train1, X_test1, y_train1, y_test1, X_train_index, X_test_index, y_train_index, y_test_index

#X_train, X_test, y_train, y_test, X_train_index, X_test_index, y_train_index, y_test_index = create_train_test_data(get_source_dataframe(), return_index=True, drop_nulls=False)
#X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_train_index.shape, X_test_index.shape, y_train_index.shape, y_test_index.shape,

In [6]:
df, retrieval_type = get_source_dataframe(folder_prefix='../../')
df_orig = df.copy()

if retrieval_type != 'tidy':
    if retrieval_type != 'raw':
        df = add_supplements(df)
        print(f'starting to save {retrieval_type} data...')
        df.to_csv(df_pathname_raw)
        print(f'finished saving {retrieval_type} data!')
    df = tidy_dataset(df, version=int(VERSION))
    df = feature_engineer(df, version=int(VERSION))

    df = df[columns]

    print(f'starting to save {retrieval_type} data...')
    df.to_csv(df_pathname_tidy)
    print(f'finished saving {retrieval_type} data!')

print(df.shape)
df[:5]

df_orig.merge(df, how='inner', left_index=True, right_index=True)

print(df.index)

loaded data from ../../data/final/df_listings_v03.csv
(51501, 7)
Int64Index([ 14520525,  27953107,  33593487,  35271294,  35429088,  44749111,
             46204665,  49020666,  49036279,  49303873,
            ...
            126173423, 126173600, 126175973, 126178769, 126179018, 126179672,
            126180107, 126180704, 126180962, 126181118],
           dtype='int64', length=51501)


In [7]:
df

,Price,bedrooms,bathrooms,nearestStation,latitude_deviation,longitude_deviation,tenure.tenureType
14520525,550000.0,3.0,1.0,0.274316,0.067288,0.141915,LEASEHOLD
27953107,400000.0,2.0,2.0,0.305845,0.086728,0.417495,LEASEHOLD
33593487,579950.0,2.0,1.0,0.438045,0.015482,0.273665,FREEHOLD
35271294,370000.0,2.0,1.0,0.399307,0.013094,0.075049,LEASEHOLD
35429088,599950.0,2.0,1.0,0.238187,0.114368,0.076125,NaN
...,...,...,...,...,...,...,...
126179672,600000.0,3.0,2.0,0.545665,0.105492,0.009635,LEASEHOLD
126180107,419999.0,2.0,1.0,0.191407,0.068753,0.012141,LEASEHOLD
126180704,475000.0,2.0,1.0,0.308609,0.080479,0.076603,LEASEHOLD
126180962,450000.0,NaN,1.0,0.476935,0.129443,0.056872,FREEHOLD


In [8]:
df.isnull().sum()

Price                     0
bedrooms               1714
bathrooms              3149
nearestStation            0
latitude_deviation       10
longitude_deviation       0
tenure.tenureType      3352
dtype: int64

In [9]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51501 entries, 14520525 to 126181118
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                51501 non-null  float64
 1   bedrooms             49787 non-null  float64
 2   bathrooms            48352 non-null  float64
 3   nearestStation       51501 non-null  float64
 4   latitude_deviation   51491 non-null  float64
 5   longitude_deviation  51501 non-null  float64
 6   tenure.tenureType    48149 non-null  object 
dtypes: float64(6), object(1)
memory usage: 5.2+ MB


,Price,bedrooms,bathrooms,nearestStation,latitude_deviation,longitude_deviation
count,51501.000000,49787.000000,48352.000000,51501.000000,5.149100e+04,5.150100e+04
mean,420845.464438,11.838693,1.177159,0.445829,6.952685e-02,1.391510e-01
std,109595.522981,2196.020360,0.410369,1.071443,4.656998e-02,7.234829e-01
min,100000.000000,1.000000,1.000000,0.000000,4.000000e-07,2.000000e-07
25%,335000.000000,1.000000,1.000000,0.224756,3.113350e-02,4.650480e-02
50%,425000.000000,2.000000,1.000000,0.364523,6.375360e-02,1.033192e-01
75%,500000.000000,3.000000,1.000000,0.558390,1.010514e-01,1.976652e-01
max,600000.000000,490000.000000,12.000000,192.431869,2.205226e-01,5.162385e+01


In [10]:
old_length = len(df)
df['location.latitude'] = pd.to_numeric(df['location.latitude'], 'coerce').dropna().astype(float)
df = df[(df['location.longitude'] <= 10)]
df = df[(df['longitude_deviation'] <= 1)]
df = df[(df['longitude_deviation2'] <= 1)]
df = df[(df['bedrooms'] <= 10)]
df = df[df['bathrooms'] <= 5]
df = df[(df['nearestStation'] <= 20)]

print(f"dataframe contract due to cleaning: {old_length} ==> {len(df)}")
old_length = len(df)

df.describe().T

KeyError: 'location.latitude'

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()
print(f"{old_length} ==> {len(df)}")
old_length = len(df)
df.describe()

In [ ]:
X_train, X_test, y_train, y_test, X_train_index, X_test_index, y_train_index, y_test_index = create_train_test_data(df,
                                                                                                                    return_index=True,
                                                                                                                    drop_nulls=True)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_train_index.shape, X_test_index.shape,
      y_train_index.shape, y_test_index.shape)
#print(type(X_train))
#X_train[0]

In [ ]:
#imputer = SimpleImputer(strategy='mean')
#imputer.fit(X_train[6])
#X_train[6] = imputer.transform(X_train[6])

In [ ]:
from time import time

pipe.fit(X_train, y_train)

model = Ridge()
model.get_params()

In [ ]:

from sklearn.pipeline import Pipeline

# Best Score:  0.30582573121661794
# Best Score:  {'alpha': 10, 'fit_intercept': True, 'max_iter': 1000, 'positive': False, 'selection': 'cyclic', 'tol': 0.001, 'warm_start': True}
# Best Score:  Lasso(alpha=10, tol=0.001, warm_start=True)
# Best Score:  138

# find optimal alpha with grid search
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#alpha = [1, 10, 100]
fit_intercept = [True, False]
max_iter = [100, 1000, 10000]
positive = [True, False]
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
tol = [0.00001, 0.0001, 0.001, 0.01]
warm_start = [True, False]
# ['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'positive', 'precompute', 'random_state', 'selection', 'tol', 'warm_start'].

options__n_neighbours = [3, 5, 7, 9, 15, 31]
options__leafsize = [2, 3, 4, 57, 9, 13, 21]

param_grid = dict(model__alpha=alpha, model__fit_intercept=fit_intercept, model__max_iter=max_iter,
                  model__positive=positive,
                  model__tol=tol, model__solver=solver)
#param_grid = dict(estimator__n_neighbors=options__n_neighbours, estimator__leaf_size= options__leafsize)

#param_grid = {'model__n_neighbors': options__n_neighbours,'model__leaf_size': options__leafsize},
#param_grid = {'n_neighbors': options__n_neighbours,                   'leaf_size': options__leafsize},
cv = 2
n_jobs = 1
verbose = 1
refit = True

#grid = RandomizedSearchCV(estimator=model, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)

gs = RandomizedSearchCV(pipe, param_grid, cv=cv, n_jobs=n_jobs,
                        verbose=verbose, scoring=CROSS_VALIDATION_SCORING, refit=refit,
                        return_train_score=True),
gs

grid_result = gs[0].fit(X_train, y_train)


In [ ]:
pipe = grid_result.best_estimator_
timings = []

t0 = time()
pipe.fit(X_train, y_train)
timings.append(time() - t0)

print(timings)
average_time = sum(timings) / len(timings)
print(average_time)

In [ ]:
def print_results(results):
    print(f'BEST PARAMS: {results.best_params_}')

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print(f'{round(mean, 3)} (+/-{round(std * 2, 3)}) for {params}')


print_results(grid_result)
print('Best Index: ', grid_result.best_index_)
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)
#print('Best Model: ', grid_result.)
#print('Best Params: ', grid_result.best_params_)[out]
### Best Score:  0.4883436188936269
### Best Params:  {'alpha': 0.01}


In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

y_pred = y_pred.reshape((-1, 1))

R2 = r2_score(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = math.sqrt(MSE)
print('-' * 10 + ALGORITHM + '-' * 10)
print('R square Accuracy', R2)
print('Mean Absolute Error Accuracy', MAE)
print('Mean Squared Error Accuracy', MSE)
print('Root Mean Squared Error', RMSE)

In [ ]:
if debug_mode:
    print(y_test_index.reshape((-1, 1)).shape);
    print(y_pred.reshape((-1, 1)).shape);
    print(y_test.shape);
    print(y_test_index.shape);
    print(y_pred.shape);
    print(y_test.shape)

In [ ]:
compare = np.hstack((y_test_index, y_test, y_pred))
compare_df = DataFrame(compare, columns=['reference', 'actual', 'predicted'])
compare_df['difference'] = abs(compare_df['actual'] - compare_df['predicted'])
compare_df['diff 1 %'] = abs((compare_df['actual'] - compare_df['predicted']) / compare_df['actual'] * 100)
compare_df['diff 2 %'] = abs((compare_df['actual'] - compare_df['predicted']) / compare_df['predicted']) * 100
compare_df['reference'] = compare_df['reference'].astype(str)
compare_df.set_index('reference', inplace=True)
compare_df

In [ ]:
compare_df.merge(df[columns], how='inner', left_index=True, right_index=True).sort_values(['diff 1 %'], ascending=False)

Notes:
* 96587218 has been removed from sale
* 117356900 is a hotel room
* 125520530

In [ ]:
score = pipe.score(X_test, y_test)
score

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.scatter(y_test, pipe.predict(X_test), edgecolors=(0, 0, 1))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
ax.set_ylabel('Predicted')
ax.set_xlabel('Actual')
#ax.title.set_text(f'CV Chosen best option ({calculated_best_pipe[1]})')
plt.show()

In [ ]:
from datetime import datetime

results = {
    'Score': score,
    'R square Accuracy': R2,
    'Mean Absolute Error Accuracy': MAE,
    'Mean Squared Error Accuracy': MSE,
    'Root Mean Squared Error': RMSE,
    'Training Time': average_time,
    'random_state': RANDOM_STATE,
    'date': str(datetime.now()),
    'params': grid_result.best_params_
}
import json


def get_results():
    results_filename = '../../results/results.json'

    with open(results_filename) as f:
        raw_audit = f.read()
    results_json = json.loads(raw_audit)
    return results_json


def update_results(saved_results_json, new_results):
    key = f'{ALGORITHM} - {ALGORITHM_DETAIL} (v{VERSION})'.lower()
    try:
        first_run_date = str(datetime.now())
        first_run_date = saved_results_json[key]['date']
        first_run_date = saved_results_json[key]['first run']
    except:
        pass

    try:
        max_score = -1000
        max_params = 'NOT APPLICABLE'
        max_score = saved_results_json[key]['Score']
        max_params = saved_results_json[key]['params']
        max_score = saved_results_json[key]['max score']
        max_params = saved_results_json[key]['max params']
    except:
        pass

    results['first run'] = first_run_date
    if key not in saved_results_json:
        new_results['max params'] = new_results['params']
        new_results['max score'] = new_results['Score']
        new_results['suboptimal'] = 'pending'
    elif max_score > saved_results_json[key]['Score']:
        new_results['suboptimal'] = 'suboptimal'
    elif max_score == saved_results_json[key]['Score']:
        if saved_results_json[key]['params'] != new_results['params']:
            new_results['max params'] = 'MULTIPLE PARAM OPTIONS'
        else:
            new_results['max params'] = saved_results_json[key]['params']
            new_results['max score'] = saved_results_json[key]['Score']
            new_results['suboptimal'] = 'pending'
    else:
        new_results['max params'] = saved_results_json[key]['params']
        new_results['max score'] = saved_results_json[key]['Score']
        new_results['suboptimal'] = 'pending'

    saved_results_json[key] = new_results

    results_filename = '../../results/results.json'
    with open(results_filename, 'w') as file:
        file.write(json.dumps(saved_results_json, indent=4))


if not IN_COLAB:
    results_json = get_results()
    update_results(results_json, results)

results

In [ ]:
deep_dive_csv = pd.read_csv(df_pathname_raw, index_col=0)

In [ ]:
deep_dive_csv.loc[125520530, ['Price', 'keyFeatures', 'sharedOwnership']]
deep_dive_csv.loc[85795281, ['Price', 'keyFeatures', 'sharedOwnership']]

deep_dive_csv.loc[85795281, 'text.description']


In [ ]:
deep_dive_csv.loc[85795281]

In [ ]:
property_dataset = deep_dive_csv.loc[[125520530, 96587218], :]
property_dataset

(
        (property_dataset['sharedOwnership.sharedOwnership'] == True) |
        (property_dataset['analyticsProperty.priceQualifier'] == 'Shared ownership') |
        (property_dataset['keyFeatures'].str.contains('shared ownership'))
)

(property_dataset['keyFeatures'].str.contains('%'))

#(property_dataset['sharedOwnership'])
#'share' isin (property_dataset['keyFeatures'].str.lower())

#df[df.beer_style.str.contains('IPA')]
property_dataset[property_dataset.keyFeatures.str.contains('IPA')]

#(property_dataset['keyFeatures'].str.contains('%')))